In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
import json

In [2]:
def extract_json(filename):
    json_file_path = filename
    parser = []
    
    with open(json_file_path, "r") as file:
        data = json.load(file)
    
    for file in data:
        filename = os.path.join(file["uuid"] + ".png")
        imgwidth = file["width"]
        imgheight = file["height"]
        
        for box in file["boundingBoxes"]:
            name = box["concept"]
    
            boxx = box["x"]
            boxy = box["y"]
            boxwidth = box["width"]
            boxheight = box["height"]
            
            parser.append([filename, imgwidth, imgheight, name, boxwidth, boxheight, boxx, boxy])

    return parser

In [3]:
df = pd.DataFrame(extract_json("Ophiuroidea.json"), columns = ["filename", "imgwidth", "imgheight", "name", "boxwidth", "boxheight", "boxx", "boxy"])

In [80]:
df.info()
df["name"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   filename   2 non-null      object 
 1   imgwidth   2 non-null      int64  
 2   imgheight  2 non-null      int64  
 3   name       2 non-null      object 
 4   boxwidth   2 non-null      int64  
 5   boxheight  2 non-null      int64  
 6   boxx       2 non-null      int64  
 7   boxy       2 non-null      int64  
 8   center_x   2 non-null      float64
 9   center_y   2 non-null      float64
 10  w          2 non-null      float64
 11  h          2 non-null      float64
dtypes: float64(4), int64(6), object(2)
memory usage: 324.0+ bytes


name
Plexauridae    1
Ophiuroidea    1
Name: count, dtype: int64

In [22]:
columns = ["filename", "imgwidth", "imgheight", "name", "boxwidth", "boxheight", "boxx", "boxy"]

df["center_x"] = ((df["boxwidth"] / 2 + df["boxx"]) / df["imgwidth"])
df["center_y"] = ((df["boxheight"] / 2 + df["boxy"]) / df["imgheight"])

df["w"] = df["boxwidth"] / df["imgwidth"]
df["h"] = df["boxheight"] / df["imgheight"]
df.head()

,filename,imgwidth,imgheight,name,boxwidth,boxheight,boxx,boxy,center_x,center_y,w,h
0,b0e7924d-53c4-4560-a041-d810195da943.png,1600,900,Plexauridae,1613,916,0,0,0.504062,0.508889,1.008125,1.017778
1,b0e7924d-53c4-4560-a041-d810195da943.png,1600,900,Ophiuroidea,1496,900,0,0,0.467500,0.500000,0.935000,1.000000


In [78]:
images = df['filename'].unique()
len(images)

In [60]:
img_df = pd.DataFrame(images, columns=['filename'])

In [61]:
img_train = tuple(img_df.sample(frac=0.8)['filename'])
#shuffling data and picking 80% of images

In [79]:
#picking all images that were not in the 80% train set (the other 20%)
img_test = tuple(img_df[~img_df['filename'].isin(img_train)]['filename'])
len(img_train), len(img_test) #length of each set

(1, 0)

In [74]:
#making dataframes
train_df = df.query(f'filename in {img_train}') 
test_df = df.query(f'filename in {img_test}')

In [77]:
train_df.head()

,filename,imgwidth,imgheight,name,boxwidth,boxheight,boxx,boxy,center_x,center_y,w,h


In [ ]:
test_df.head()

In [ ]:
#Converting species names into label encoding (IDs)
